# EdgePairCoverage EPC

## Parameter setting

Setting of all parameters

In [1]:
regexStr='a+b+'#Regular expressions to test
accChars=[]#This parameter is no longer used

## Regex -> DFA that accepts its complement

### Java Kernel

Load Java package dk.brics.automaton via jpype

In [2]:
import copy
import random

In [3]:
import jpype
import os
jarpath = os.path.join(os.path.abspath("."), "dkbrics.jar")
jvmPath = jpype.getDefaultJVMPath()
jpype.startJVM(jvmPath,"-ea", "-Djava.class.path=%s" % (jarpath))
RegExp = jpype.JClass("dk.brics.automaton.RegExp")

Generating a minimal deterministic automaton that accepts its complement by means of a regular expression and converting it into string form for subsequent processing

In [4]:
regex = RegExp(regexStr)
automaton=regex.toAutomaton()
automaton.minimize()
dfa=automaton.toString()
# dfs=automaton.toDot()
# print(dfa)
dfastr=str(dfa)
# dfastr[0:9]=='singleton'
# jpype.shutdownJVM()

### Python Data Processing

Class-automaton definitions

In [5]:
class transition:
    def __init__(self, char,toward):
        self.char =char
        self.toward=toward
class point:#point include 0 to multiple transitions
    def __init__(self, state):
        self.state = state
        self.trans = []
class automata:#automata include 0 to multiple points
    def __init__(self):
        self.points = []
        self.init=-1
        self.acc = []

method getCharList: finds the characters in the alphabet of the given sequence, returns list

In [6]:
def getCharList(start,end,accept=[]):
    asciiCode=[ '  ',  '!',  '" ',  '#',  '$',  '%',  '&',  '\' ',  '( ',  ')',  '*',  '+',  ',',  '-',  '.',  '/',  '0',  '1',  '2',  '3',  '4',  '5',  '6',  '7',  '8',  '9',  ':',  ';',  '< ',  '= ',  '> ',  '?',  '@',  'A',  'B',  'C',  'D',  'E',  'F',  'G',  'H',  'I',  'J',  'K',  'L',  'M',  'N',  'O',  'P',  'Q',  'R',  'S',  'T',  'U',  'V',  'W',  'X',  'Y',  'Z',  '[ ',  '\\',  ']',  '^',  '_',  '`',  'a',  'b',  'c',  'd',  'e',  'f',  'g',  'h',  'i',  'j',  'k',  'l',  'm',  'n',  'o',  'p',  'q',  'r',  's',  't',  'u',  'v',  'w',  'x',  'y',  'z',  '{ ',  '|',  '}',  '~']
    if len(accept)==0:
        accept=asciiCode
    charlist=[]
    char=start
    endplus=chr(ord(end)+1)
    while char!=endplus :
        if char in accept:
            charlist.append(char)
        char=chr(ord(char)+1) 
    return charlist

method checkCharList: returns a list of accepted characters by means of a range of characters given as a string

In [7]:
def checkCharList(charstr,accept=[]):
    chars=[]
    if charstr.find('-')>=0 and len(charstr)>1:
        if len(charstr.split('-'))==3 and charstr.split('-')[0]=='':
            a='-'
            b=eval(repr(charstr.split('-')[2]).replace('\\\\', '\\'))
        elif len(charstr.split('-'))==3 and charstr.split('-')[2]=='':
            b='-'
            a=eval(repr(charstr.split('-')[0]).replace('\\\\', '\\'))
        else:
            a=eval(repr(charstr.split('-')[0]).replace('\\\\','\\'))
            b=eval(repr(charstr.split('-')[1]).replace('\\\\', '\\'))
        chars.extend(getCharList(a,b,accept))
    else:
        chars.append(charstr)
    return chars

Generating defined automaton DFAs via Java package generated DFAs

In [8]:
auto=automata()
dfaList=dfastr.split('\n')
auto.init=int(dfaList[0].split(': ')[1])
del dfaList[0]
for i in dfaList:
    if i[0:5]=='state':
        state_new=int(i.split(' ')[1])
        if state_new!=0:
            auto.points.append(point_new)
        point_new=point(state_new)
        if i.split(' ')[2]=='[accept]:':
            auto.acc.append(state_new)
        #print(i.split(' '))
    elif i=='':
        auto.points.append(point_new)
    else:
        temp=i.split(' -> ' )
        charString=temp[0].strip()
        charList=checkCharList(charString,accChars)
        for theChar in charList:
            transition_new=transition(theChar,int(temp[1]))
            point_new.trans.append(transition_new)

Read all characters to complete the DFA

In [9]:
allTransChars=[]
for Point in auto.points:
    for Transition in Point.trans:
        allTransChars.append(Transition.char)
accChars=list(set(allTransChars))

Adding a DEAD status

In [10]:
point_dead=point(-1)
auto.points.append(point_dead)

Complementary connection layer

In [11]:
for Point in auto.points:
    TransChars=[]
    for Transition in Point.trans:
        TransChars.append(Transition.char)
    if set(TransChars)!=set(accChars):
        #print("lack:",Point.state)
        for thisChar in accChars:
            if thisChar not in TransChars:
                #print(thisChar," not in ")#,TransChars)
                transition_new=transition(thisChar,int(-1))
                Point.trans.append(transition_new)

Reconciliation of acceptance and rejection states

In [12]:
States=[]
for Point in auto.points:
    States.append(Point.state)
auto.acc=list(set(States)-set(auto.acc))

Streamlined automaton with one random path reserved between two states (optional)

In [13]:
def LiteAutomata(auto):
    for Point in auto.points:
        Towards=[]
        newTrans=[]
        for Transition in Point.trans:
            if Transition.toward not in Towards:
                Towards.append(Transition.toward)
        for Target in Towards:
            alllist=[]
            for Transition in Point.trans:
                if Transition.toward == Target:
                    alllist.append(Transition)
            if len(alllist)>1:
                length=len(alllist)
                replist=[]
                randomValue=random.sample(range(0,len(alllist)),1)[0]
                replist.append(alllist[randomValue])
                newTrans.extend(replist)
            else:
                newTrans.extend(alllist)
        Point.trans=newTrans

Delete this line if you don't want to streamline the automaton

In [14]:
LiteAutomata(auto)

## DFA's Edge-pair Coverage

### Data pre-processing

Definition of class edge-pairs

In [15]:
class edgePair:
    def __init__(self, state1,char1to2,state2,char2to3,state3):
        self.state1 =state1
        self.char1to2=char1to2
        self.state2 =state2
        self.char2to3=char2to3
        self.state3 =state3
        self.visited=False

Class - Point definitions for edge pair-based proximity tables

In [16]:
class edgePairPoint:
    def __init__(self, state):
        self.state =state
        self.edgePairs=[]

Generating edge-pair based proximity tables

In [17]:
adjList=[]
for Point in auto.points:
    new_epp=edgePairPoint(Point.state)
    adjList.append(new_epp)

method getEPoint: returns the point with the state State from the adjList

In [18]:
def getEPPoint(State,adjList):
    for epp in adjList:
        if epp.state==State:
            break
    return epp

method getPoint: returns the point with the state State from the automaton

In [19]:
def getPoint(State,auto):
    for point in auto.points:
        if point.state==State:
            break
    return point

Generate a collection of edge-pairs edgePairs

In [20]:
for Point in auto.points:
    for Transition in Point.trans:
        nextPoint=getPoint(Transition.toward,auto)
        for nextTransition in nextPoint.trans:
            newEdgePair= edgePair(Point.state, Transition.char, Transition.toward, nextTransition.char, nextTransition.toward)
            epp = getEPPoint(Point.state,adjList)
            epp.edgePairs.append(newEdgePair)

### Edge pair coverage algorithm

Depth-first search algorithm based on edge pair indexing to generate depth-first search paths

In [21]:
def detectRingEP(epp):
    for EdgePair in epp.edgePairs:
        if epp.state==EdgePair.state3 and EdgePair.visited==False:
            return EdgePair
    return False
def tagVisitEP(state1,char1to2,state2,char2to3,state3,adjList):
    for epp in adjList:
        if epp.state==state1:
            for EdgePair in epp.edgePairs:
                if EdgePair.state2==state2\
                and EdgePair.char1to2==char1to2\
                and EdgePair.state3==state3\
                and EdgePair.char2to3==char2to3:
                    EdgePair.visited=True
def tagVisitRoad(each,adjList):
    for n in range(0,len(each)-1):
        tagVisitEP(each[n].state2,each[n].char2to3,each[n+1].state1,each[n+1].char1to2,each[n+1].state2,adjList)
class Ring:
    def __init__(self, edgePair):
        self.edgePair =edgePair
        self.repeat=False
def getRingRoad(EdgePair,Rings):
    for ring in Rings:
        if EdgePair.state1==ring.edgePair.state1 \
        and EdgePair.char1to2==ring.edgePair.char1to2 \
        and EdgePair.state2==ring.edgePair.state2 \
        and EdgePair.char2to3==ring.edgePair.char2to3 \
        and EdgePair.state3==ring.edgePair.state3:
            return ring
def clearRingRoad(edgePairCoverage):
    RingsEP=[]
    Rings=[]
    for road in edgePairCoverage:
        for EdgePair in road:
            if EdgePair.state1==EdgePair.state3:
                if EdgePair not in RingsEP:
                    RingsEP.append(copy.deepcopy(EdgePair))
    for EdgePair in RingsEP:
        new_ring=Ring(EdgePair)
        Rings.append(new_ring)
    for road in edgePairCoverage:
        for EdgePair in road:
            if EdgePair.state1==EdgePair.state3:
                #print(EdgePair.state1,EdgePair.state3)
                theRing=getRingRoad(EdgePair,Rings)
                if theRing.repeat==False:
                    #print('tag1')
                    theRing.repeat=True
                else:
                    road.remove(EdgePair)
                    #print('clear1')
def DFS(epp,road,adjList,edgePairCoverage):
    #tagVisitRoad(road,adjList)
    flag=False
    EdgePair=detectRingEP(epp)
    if EdgePair and len(road)==0:
        road.append(EdgePair)
        EdgePair.visited=True
        flag=True
        DFS(epp,road,adjList,edgePairCoverage)
    elif EdgePair and road[-1].state3==EdgePair.state2:
        road.append(EdgePair)
        EdgePair.visited=True
        flag=True
        DFS(epp,road,adjList,edgePairCoverage)
    else:
        for EdgePair in epp.edgePairs:
            if EdgePair.visited==False:
                road.append(EdgePair)
                EdgePair.visited=True
                nextPoint=getEPPoint(EdgePair.state2,adjList)
                DFS(nextPoint,road,adjList,edgePairCoverage)
                flag=True
    if len(road)>0:
        if flag==False:
            edgePairCoverage.append(copy.deepcopy(road))#Cannot move on, storage path
            #clearRingRoad(road)
        road.pop()
def DFSearch(epp,road,adjList,edgePairCoverage):
    DFS(epp,road,adjList,edgePairCoverage)
    for sp in adjList:
        DFS(sp,road,adjList,edgePairCoverage)
    clearRingRoad(edgePairCoverage)

Generating a set of edge pairs overlays

In [22]:
road=[]
edgePairCoverage=[]
startPoint=getEPPoint(auto.init,adjList)
DFSearch(startPoint,road,adjList,edgePairCoverage)

Definition of class-edge

In [23]:
class edge:
    def __init__(self, state1,char1to2,state2):
        self.state1 =state1
        self.char1to2=char1to2
        self.state2 =state2

Dump the path of an edge pair into the path of an edge and store it in the list edgeRoads

In [24]:
edgeRoads=[]
for each in edgePairCoverage:
    edgeRoad=[]
    for EdgePair in each:
        new_edge1=edge(EdgePair.state1,EdgePair.char1to2,EdgePair.state2)
        edgeRoad.append(new_edge1)
    new_edge2=edge(EdgePair.state2,EdgePair.char2to3,EdgePair.state3)
    edgeRoad.append(new_edge2)
    edgeRoads.append(copy.deepcopy(edgeRoad))

method onMove: for edges that do not touch the final state, extends them by the shortest path to the terminated state

In [25]:
import heapq
import math

def generateEdgeAdjList(auto):
    G={}
    for Point in auto.points:
        G[Point.state]={}
        for Transition in Point.trans:
            G[Point.state][Transition.toward]=1
        G[Point.state][Point.state]=0
    return G

def init_distance(graph, s):
    distance = {s: 0}
    for vertex in graph:
        if vertex != s:
            distance[vertex] = math.inf
    return distance

def dijkstra(graph, s):
    pqueue = []
    heapq.heappush(pqueue, (0, s))
    seen = set()
    parent = {s: None}
    distance = init_distance(graph, s)
    while len(pqueue) > 0:
        pair = heapq.heappop(pqueue)
        dist = pair[0]
        vertex = pair[1]
        seen.add(s)
        nodes = graph[vertex].keys()
        for w in nodes:
            if w not in seen:
                if dist + graph[vertex][w] < distance[w]:
                    heapq.heappush(pqueue, (dist + graph[vertex][w], w))
                    parent[w] = vertex
                    distance[w] = dist + graph[vertex][w]
    return parent, distance

def getFirstEdge(state1,state2,auto):
    thisPoint=getPoint(state1,auto)
    for Transition in thisPoint.trans:
        if Transition.toward==state2:
            break
    new_edge=edge(state1,Transition.char,state2)
    return new_edge

def onMove(edgeRoad,auto,G):
    if edgeRoad[-1].state2 in auto.acc:
        return# edgeRoad
    else:
        parent_dict, distance_dict =dijkstra(G, edgeRoad[-1].state2)
        shortestAcc=-1
        minDistance=math.inf
        for Toward in distance_dict.keys():
            if distance_dict[Toward]<minDistance and Toward in auto.acc:
                shortestAcc=Toward
                minDistance=distance_dict[Toward]
        reachEdges=[]
        thisState=shortestAcc
        while parent_dict[thisState]!=None:
            new_edge=getFirstEdge(parent_dict[thisState],thisState,auto)
            reachEdges.append(new_edge)
            thisState=parent_dict[thisState]
        reachEdges.reverse()
        edgeRoad.extend(reachEdges)
        return

method onBack: for edges that do not start from the initial state, extend them to the initial state by the shortest path

In [26]:
def generateEdgeInAdjList(auto):
    G={}
    for Point in auto.points:
        G[Point.state]={}
    for Point in auto.points:
        for Transition in Point.trans:
            G[Transition.toward][Point.state]=1
        G[Point.state][Point.state]=0
    return G
def onBack(edgeRoad,auto, InG):
    if edgeRoad[0].state1 == auto.init:
        return
    else:
        parent_dict, distance_dict =dijkstra(InG, edgeRoad[0].state1)
        #print(parent_dict)
        reachEdges=[]
        thisState=auto.init
        while parent_dict[thisState]!=None:
            #print(parent_dict[thisState])
            new_edge=getFirstEdge(thisState,parent_dict[thisState],auto)
            reachEdges.append(new_edge)
            thisState=parent_dict[thisState]
        reachEdges.reverse()
        for reachEdge in reachEdges:
            edgeRoad.insert(0 , reachEdge)
        return

method onExtend: adds a path to a final state that has been traversed but not stopped

In [27]:
def onExtend(edgeRoads,auto):
    CoveredAcc=[]
    for eachRoad in edgeRoads:
        if eachRoad[-1].state2 not in CoveredAcc:
            CoveredAcc.append(eachRoad[-1].state2)
    for eachRoad in edgeRoads:
        thisRoad=copy.deepcopy(eachRoad)
        while len(thisRoad)>0:
            thisRoad.pop()
            if len(thisRoad)>0:
                if thisRoad[-1].state2 not in CoveredAcc and thisRoad[-1].state2 in auto.acc:
                    thatRoad=copy.deepcopy(thisRoad)
                    CoveredAcc.append(thisRoad[-1].state2)
                    #print(generateString(thatRoad),thisRoad[-1].state2)
                    edgeRoads.append(thatRoad)
    #print(CoveredAcc,auto.acc)

For paths that do not touch the final state, extend them to the final state; for paths that do not start from the initial state, extend them to the initial state; clean up paths that involve dead states

In [28]:
G=generateEdgeAdjList(auto)
InG=generateEdgeInAdjList(auto)
roadNum=len(edgeRoads)
delNum=0
for n in range(0,roadNum):
    try:
        onMove(edgeRoads[n-delNum],auto,G)
        onBack(edgeRoads[n-delNum],auto,InG)
    except:
        #print('remov')
        edgeRoads.remove(edgeRoads[n-delNum])
        delNum+=1

For final states that have been traversed but not stopped, add the path

In [29]:
onExtend(edgeRoads,auto)

## Generate strings by cover paths

Method generateString: Generates a string from the given edge path

In [30]:
def generateString(edgeRoad):
    string=''
    for Edge in edgeRoad:
        string=string+Edge.char1to2
    return string

Generate a string and display it

In [31]:
Strings=[]
NeStrings=[]
if auto.init in auto.acc:
    Strings.append('')  
for eachRoad in edgeRoads:
    #print(eachRoad[-1].state2)
    if eachRoad[-1].state2 in auto.acc:
        Strings.append(generateString(eachRoad))
    else:
        print("a positive string found")
        NeStrings.append(generateString(eachRoad))
#print(Strings)#NeStrings)

In [32]:
if auto.init in auto.acc:
    print('[Negative] (null string)')
for eachRoad in edgeRoads:
    if eachRoad[-1].state2 in auto.acc:
        print('[Negative] ',generateString(eachRoad))
    else:
        print('[Positive]',generateString(eachRoad))

[Negative] (null string)
[Negative]  aaabbbabb
[Negative]  aaba
[Negative]  aba
[Negative]  bb
[Negative]  aaa
